# 准备环境

In [1]:
from pyalink.alink import *
resetEnv()
useLocalEnv(1, config=None)


Use one of the following command to start using pyalink:
使用以下一条命令来开始使用 pyalink：
 - useLocalEnv(parallelism, flinkHome=None, config=None)
 - useRemoteEnv(host, port, parallelism, flinkHome=None, localIp="localhost", config=None)
Call resetEnv() to reset environment and switch to another.
使用 resetEnv() 来重置运行环境，并切换到另一个。

JVM listening on 127.0.0.1:64279


JavaObject id=o6

# 数据准备

In [2]:
## prepare data
import numpy as np
import pandas as pd
data = np.array([
    [0, 0.0, 0.0, 0.0],
    [1, 0.1, 0.1, 0.1],
    [2, 0.2, 0.2, 0.2],
    [3, 9, 9, 9],
    [4, 9.1, 9.1, 9.1],
    [5, 9.2, 9.2, 9.2]
])
df = pd.DataFrame({"id": data[:, 0], "f0": data[:, 1], "f1": data[:, 2], "f2": data[:, 3]})
inOp = BatchOperator.fromDataframe(df, schemaStr='id double, f0 double, f1 double, f2 double')
FEATURE_COLS = ["f0", "f1", "f2"]
VECTOR_COL = "vec"
PRED_COL = "pred"

# 数据预处理

In [3]:
vectorAssembler = (
    VectorAssembler()
    .setSelectedCols(FEATURE_COLS)
    .setOutputCol(VECTOR_COL)
)

# 聚类训练

In [4]:
kMeans = (
    KMeans()
    .setVectorCol(VECTOR_COL)
    .setK(2)
    .setPredictionCol(PRED_COL)
)

# 数据预测

In [5]:
pipeline = Pipeline().add(vectorAssembler).add(kMeans)
pipeline.fit(inOp).transform(inOp).firstN(9).collectToDataframe()

,id,f0,f1,f2,vec,pred
0,0.0,0.0,0.0,0.0,0.0 0.0 0.0,1
1,1.0,0.1,0.1,0.1,0.1 0.1 0.1,1
2,2.0,0.2,0.2,0.2,0.2 0.2 0.2,1
3,3.0,9.0,9.0,9.0,9.0 9.0 9.0,0
4,4.0,9.1,9.1,9.1,9.1 9.1 9.1,0
5,5.0,9.2,9.2,9.2,9.2 9.2 9.2,0
